In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline

In [2]:
import csv

with open('datasets/train.tsv') as f:
    rows_train = [line[:-1].split('\t') for line in f] 
    
with open('datasets/public.tsv') as f:
    rows_public = [line[:-1].split('\t') for line in f] 

In [3]:
rows_train[200:250]

[['621441536517',
  '',
  '',
  'я с ним не разговаривал .',
  '4',
  'не виделся ну , раздевайтесь .',
  'bad',
  '0.96996169'],
 ['621441536517',
  '',
  '',
  'я с ним не разговаривал .',
  '5',
  'совсем ?',
  'good',
  '0.9682806455'],
 ['630918374463',
  '',
  'двое .',
  'о . вау !',
  '0',
  'что вау ?',
  'good',
  '0.6852331573'],
 ['630918374463',
  '',
  'двое .',
  'о . вау !',
  '1',
  'скажи ?',
  'neutral',
  '0.6339007578'],
 ['630918374463',
  '',
  'двое .',
  'о . вау !',
  '2',
  'чувствуешь себя униженным ? да . меня обогнал хороший человек .',
  'bad',
  '0.8809825665'],
 ['630918374463',
  '',
  'двое .',
  'о . вау !',
  '3',
  'удивлена ?',
  'good',
  '0.6959357362'],
 ['630918374463',
  '',
  'двое .',
  'о . вау !',
  '4',
  'это банально , говорить " вау " ?',
  'good',
  '0.8890238226'],
 ['630918374463',
  '',
  'двое .',
  'о . вау !',
  '5',
  'что " ого " ?',
  'bad',
  '0.6807389284'],
 ['643528390848',
  'мой пиджак еще там .',
  'оставь его .',
  '

In [4]:
import enchant
d = enchant.Dict("en_US")

def check_en(rows, maxind=2):
    global d
    for r in tqdm(rows, position=0):
        for k in r[:len(r)-maxind]:
            if k != '' and d.check(k) and not k.isnumeric():
                print(r, k)
                return
            
check_en(rows_train)
check_en(rows_public, 0)

100%|██████████| 9968/9968 [00:01<00:00, 6243.17it/s]


видно, что нет реплик на английском.

In [6]:
def get_stats(rows):
    lens = set()
    for i,r in enumerate(rows):
        if len(r) not in lens:
            lens.add(len(r))
            print(i, len(r), r)
            
    summ = 0
    maxx = 0
    for i, r in enumerate(rows):
        summ += (len(r[1].split()) + len(r[2].split()) + len(r[3].split()) + len(r[5].split()))
        maxx = np.max([maxx] + [len(r[t].split()) for t in [1, 2, 3, 5]])

    print('mean token len: {}'.format(summ / len(rows)))
    print('max token replic {}'.format(maxx))
    

get_stats(rows_train)
print('**************')
get_stats(rows_public)

0 8 ['22579918886', 'кликни на меня а потом на надпись " видео - звонок " .', 'о , я тебя вижу .', 'ладно , повесь трубку .', '0', 'не могу .', 'good', '0.8753516175']
mean token len: 20.346170014251587
max token replic 65
**************
0 6 ['138920940977', 'знаешь , я иногда подумываю , что тебе надо принести сюда свою гитару и показать местным настоящую игру .', 'не - а .', 'нет ?', '0', 'неа .']
mean token len: 20.65730337078652
max token replic 37


в каждой строке одинаковое число \t, пустые реплики - ''. Возьмем максимум 30 токенов на реплику.

In [9]:
train_df = pd.DataFrame(rows_train)
public_df = pd.DataFrame(rows_public)

In [8]:
train_df.head()

,0,1,2,3,4,5,6,7
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.8753516175
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.9009682113
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.8843202145
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.9825304673
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.8380535096


In [10]:
public_df.head()

,0,1,2,3,4,5
0,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,0,неа .
1,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,1,"нет , не хочу ."
2,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,2,нет .
3,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,3,"конечно , нет ."
4,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,4,"разумеется , нет ."


In [11]:
train_df.to_csv('train_df')
public_df.to_csv('public_df')

In [15]:
import regex

from nltk.stem.snowball import RussianStemmer

In [ ]:
def sent2vec(sent, emb, max_len=30, emb_size=300)
    vec = np.zeros((max_len, emb_size))
    for i, t in enumerate(sent):
        if t in emb:
            vec[i] = emb[t]
    return vec

def fix_text(text):
    text = regex.sub("(?s)<ref>.+?</ref>", "", text) # remove reference links
    text = regex.sub("(?s)<[^>]+>", "", text) # remove html tags
    text = regex.sub("&[a-z]+;", "", text) # remove html entities
    text = regex.sub("(?s){{.+?}}", "", text) # remove markup tags
    text = regex.sub("(?s){.+?}", "", text) # remove markup tags
    text = regex.sub("(?s)\[\[([^]]+\|)", "", text) # remove link target strings
    text = regex.sub("(?s)\[\[([^]]+\:.+?]])", "", text) # remove media links
    text = regex.sub("[']{5}", "", text) # remove italic+bold symbols
    text = regex.sub("[']{3}", "", text) # remove bold symbols
    text = regex.sub("[']{2}", "", text) # remove italic symbols
    text = regex.sub(u"[^ \r\n\p{Cyrillic}.?!\-]", " ", text)
    text = text.lower()
    return text

def tokenize_word(word, stemmer):
    stem =  stemmer.stem(word)
    affix = word[len(stem):]

    if affix:
        return (stem, '#' + affix)  #To make embeddings work, all suffixes start with artificial token '#'
    else:
        return (stem, )

def clear_sent(sent, stemmer=RussianStemmer):
    sent = fix_text(sent)
    phrases = regex.split("([.?!])?[\n]+|[.?!] ", sent)
    words = [s.split() for s in phrases]
    print(words)
    sent = []
    for s in words:
        for w in s:
            tokens = tokenize_word(w, stemmer)
            for t in tokes:
                sent.append(t)
    
    return sent
    
    

def context2vec(context, emb, max_len=30, emb_size=300, stemmer=RussianStemmer()):
    assert len(context) == 4
    vecs = None
    for sent in context:
        sent = clear_sent(sent, stemmer)
        vec = sent2vec(sent, emb, emb_size, max_len, emb_size)
        if vecs is None:
            vecs = vec
        else:
            vecs = np.concatenate([vecs, vec], axis = 0)
    
    return vecs